ReLU activation function is one of the breakthrought in deep neural network due to its simplcity and the fact that its locals derivative equals to $1$ for non-negative value, hence prevent the vanishing gradient problem. Despite it efectiveness and simplicity, some effort on make ReLU more expressive has been made. Some effort to  generalize  ReLU are based on using a nonzero slopes $\alpha$ for negative input. As for November 2020, there are 3 variant of ReLU that do this kind of generalization. Absolute value rectification fixes $\alpha =-1$. [LeakyReLU](https://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf) fixes $\alpha$ to a small value, while [PReLU](https://arxiv.org/pdf/1502.01852.pdf) treats $\alpha$ as a learnable parameter.

Let us begin by the formal definition of Dynamic ReLU based on the [paper](https://arxiv.org/pdf/2003.10027.pdf)


**Definition** : Let  us  denote  the  traditional  or  static  ReLU  as $\textbf{y}=  max\{\textbf{x},0\}$, where $\textbf{x}$ is the input vector. For the input $\textbf{x}_c$ at the $c$ thchannel, the activation is  computed  as $y_c=  max\{x_c,0\}$. ReLU  can  be  generalized  to  a  parametric piecewise linear function $y_c= max_k\{a_k^c(x_c)+b_k^c\}$. Dynamic ReLU further extend this piecewise linear function from static to dynamic by adapting $a_k^c$, $b_k^c$ based upon all input elements $x=\{x_c\}$ as follows: $y_c=f_{\theta(x)}(x_c) =   max_{1≤k≤K}\{a^k_c(\mathbf{x})x_c+b^k_c(\mathbf{x})\}$where the coefficients $(a^k_c,b^k_c)$ are the output of a hyper function $\theta(\mathbf{x})$ as:$[a^1_1,...,a^1_C,...,a^K_1,...,a^K_C,b^1_1,...,b^1_C,...,b^K_1,\dots,b^K_C]^{\mathsf{T}}=\theta(\mathbf{x})$,where $K$ is the number of functions, and $C$ is the number of channels.

Let us rewrite vanila ReLU $\textbf{y}=  max\{\textbf{x},0\}$ to a generalized form from the definition above. It is easy to show that $K=2$ for this vanila ReLU and $\theta(\mathbf{x}) =[a^1_c(x), b^1_c(x) , a^2_c(x) , b^2_c(x)]^{\mathsf{T}} = [1,0,0,0]^{\mathsf{T}}$. Let us plug in some number to $y_c=f_{\theta(x)}(x_c)$. Let $x_c=-1$

$f_{\theta(x)}(-1) =   max_{1≤k≤2}\{a^k_c(\mathbf{x})(-1)+b^k_c(\mathbf{x})\}=a^2_c(\mathbf{x})(-1)+b^2_c(\mathbf{x})=0(-1)+0=0$




Now take a look about how to get the theta value

<img src="theta.jpg">

In [1]:
import torch
import torch.nn as nn


class Dynamic_ReLUB2D(nn.Module):
    def __init__(self, channels, reduction=4, k=2):
        super(Dynamic_ReLUB2D, self).__init__()
        self.channels = channels
        self.k = k
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction, 2 * k * channels)
        self.sigmoid = nn.Sigmoid()
        self.register_buffer('lambdas', torch.Tensor([1.] * k + [0.5] * k).float())
        self.register_buffer('init_v', torch.Tensor([1.] + [0.] * (2 * k - 1)).float())

    def compute_theta(self, x):
        theta = torch.mean(x, dim=-1)
        theta = torch.mean(theta, dim=-1)
        theta = self.fc1(theta)
        theta = self.relu(theta)
        theta = self.fc2(theta)
        theta = 2 * self.sigmoid(theta) - 1
        return theta

    def forward(self, x):
        assert x.shape[1] == self.channels
        theta = self.compute_theta(x)
        relu_coefs = theta.view(-1, self.channels, 2 * self.k) * self.lambdas + self.init_v
        x_perm = x.permute(2, 3, 0, 1).unsqueeze(-1)
        output = x_perm * relu_coefs[:, :, :self.k] + relu_coefs[:, :, self.k:]
        result = torch.max(output, dim=-1)[0].permute(2, 3, 0, 1)
        return result


In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


Then we create simple deep neural network for classifying FashionMNIST

In [3]:
class Net(nn.Module):
    def __init__(self, dynamic_relu=False):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.is_dynamic_relu = dynamic_relu
        self.relu1 = F.relu
        self.relu2 = F.relu
        if dynamic_relu:
            self.relu1 = Dynamic_ReLUB2D(32, k=3)
            self.relu2 = Dynamic_ReLUB2D(64, k=3)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [4]:
def train(model, device, train_loader, optimizer, epoch, dry_run, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
def main(dynamic_relu=False):
    # Hyperparameters
    batch_size = 64
    test_batch_size = 1000
    epochs = 14
    lr = 1.0
    gamma = 0.7
    no_cuda = False
    dry_run = False
    seed = 1
    log_interval = 10
    save_model = False

    use_cuda = not no_cuda and torch.cuda.is_available()

    torch.manual_seed(seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    dataset1 = datasets.FashionMNIST('../data', train=True, download=True,
                                     transform=transform)
    dataset2 = datasets.FashionMNIST('../data', train=False,
                                     transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net(dynamic_relu=dynamic_relu).to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch,dry_run,log_interval)
        test(model, device, test_loader)
        scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


In [7]:
main(False)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316559
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.380474
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.340337
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.820944
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.569544
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.667861
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.707018
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.607873
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.724158
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.651603
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.556938
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.615474
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.584760
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.722667
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.609791
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.702027
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.439597
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.404360
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.542324
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.501415
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.341231
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.137763
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.222133
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.286344
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.169401
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.475920
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.317094
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.391357
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.203365
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.395156
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.341984
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.290291
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.288454
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.284616
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.296142
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.458732
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.209684
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.432227
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.333370
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.283346


Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.259919
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.122554
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.222270
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.243383
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.131784
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.212244
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.196335
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.162669
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.221498
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.316345
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.309415
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.138156
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.207623
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.237718
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.075013
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.326771
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.295490
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.122531
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.420704
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.177206


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.234256
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.240700
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.200062
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.138840
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.265478
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.259522
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.174174
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.154665
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.232619
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.406943
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.190126
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.131507
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.509736
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.236977
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.050752
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.328494
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.150578
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.102249
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.181610
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.070573


Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.216231
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.290146

Test set: Average loss: 0.2145, Accuracy: 9253/10000 (93%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.307676
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.062891
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.186545
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.223119
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.139377
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.223419
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.266077
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.118937
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.217204
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.168254
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.101391
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.185110
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.122725
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.135192
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.153845
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.157729
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.138147
Train Epoch: 8 [10

Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.149776
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.242193
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.311498
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.066394
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.209481
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.129382
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.379446
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.097715
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.123075
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.110952
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.101951
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.340416
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.167565
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.200342
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.261547
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.200623
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.078167
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.330518
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.084905
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.226916


Train Epoch: 11 [24960/60000 (42%)]	Loss: 0.088598
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.145373
Train Epoch: 11 [26240/60000 (44%)]	Loss: 0.077536
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.166923
Train Epoch: 11 [27520/60000 (46%)]	Loss: 0.268232
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.091142
Train Epoch: 11 [28800/60000 (48%)]	Loss: 0.057716
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.152954
Train Epoch: 11 [30080/60000 (50%)]	Loss: 0.186845
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.137001
Train Epoch: 11 [31360/60000 (52%)]	Loss: 0.149024
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.179782
Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.150168
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.158032
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.237828
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.174560
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.179867
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.164781
Train Epoch: 11 [36480/60000 (61%)]	Loss: 0.118108
Train Epoch: 11 [37120/60000 (6

Train Epoch: 13 [7040/60000 (12%)]	Loss: 0.195358
Train Epoch: 13 [7680/60000 (13%)]	Loss: 0.157696
Train Epoch: 13 [8320/60000 (14%)]	Loss: 0.274349
Train Epoch: 13 [8960/60000 (15%)]	Loss: 0.085156
Train Epoch: 13 [9600/60000 (16%)]	Loss: 0.292765
Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.144831
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.258484
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.297928
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.184664
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.151594
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.122511
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.220752
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.291823
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.268108
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.166741
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.110340
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.172390
Train Epoch: 13 [17920/60000 (30%)]	Loss: 0.142664
Train Epoch: 13 [18560/60000 (31%)]	Loss: 0.071799
Train Epoch: 13 [19200/60000 (32%)]	

Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.300336
Train Epoch: 14 [50560/60000 (84%)]	Loss: 0.178487
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.226012
Train Epoch: 14 [51840/60000 (86%)]	Loss: 0.232877
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.179635
Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.135269
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.149386
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.220985
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.157992
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.196811
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.087439
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.092019
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.135213
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.149708
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.198996
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.198016

Test set: Average loss: 0.2079, Accuracy: 9284/10000 (93%)



In [ ]:
main(True)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.326356
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.764864
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.072196
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.952992
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.625305
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.657928
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.845934
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.705759
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.531791
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.714365
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.613976
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.595658
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.554837
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.530920
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.782818
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.311544
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.506787
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.773301
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.643794
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.451967
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.409943
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.161115
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.364023
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.167326
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.263311
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.337780
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.568568
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.144081
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.284556
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.209575
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.223668
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.346513
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.290086
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.288176
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.316602
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.368156
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.313691
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.257476
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.264904
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.317178


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.172564
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.427440
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.293239
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.301305
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.122138
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.255386
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.252229
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.194352
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.200502
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.251890
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.133274
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.215895
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.187370
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.271824
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.275101
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.137444
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.193790
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.394230
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.257916
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.148642
